In [88]:
import warnings
warnings.filterwarnings('ignore')

In [89]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Load in CSV data files - Regular and Playoffs Games

    - Join the two data sets to create on large dataframe

In [90]:
file_path = Path('../Resources')

# Load the regular season data
reg_df = pd.read_csv(f"{file_path}/raptors_regulars.csv")

# Load the playoff season data
playoff_df = pd.read_csv(f"{file_path}/raptorsplayoffs.csv")


In [92]:
# Inspect columns and data types for data

print(reg_df.dtypes)

print(playoff_df.dtypes)

TEAM        object
DATE        object
MATCHUP     object
W/L         object
MIN          int64
PTS          int64
FGM          int64
FGA          int64
FG%        float64
3PM          int64
3PA          int64
3P%        float64
FTM          int64
FTA          int64
FT%        float64
OREB         int64
DREB         int64
REB          int64
AST          int64
STL          int64
BLK          int64
TOV          int64
PF           int64
+/-          int64
dtype: object
TEAM        object
DATE        object
MATCHUP     object
W/L         object
MIN          int64
PTS          int64
FGM          int64
FGA          int64
FG%        float64
3PM          int64
3PA          int64
3P%        float64
FTM          int64
FTA          int64
FT%        float64
OREB         int64
DREB         int64
REB          int64
AST          int64
STL          int64
BLK          int64
TOV          int64
PF           int64
+/-          int64
dtype: object


In [93]:
# Join the 2 datasets using concat()

joined_df = pd.concat([reg_df,playoff_df], ignore_index=True)

joined_df.head(3)

,TEAM,DATE,MATCHUP,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,TOR,01/13/2019,TOR @ WAS,W,290,140,49,104,47.1,13,...,80.6,17,42,59,24,16,8,21,27,2
1,TOR,11/29/2018,TOR vs. GSW,W,265,131,47,90,52.2,15,...,91.7,7,30,37,25,5,6,14,23,3
2,TOR,02/13/2019,TOR vs. WAS,W,241,129,44,92,47.8,16,...,83.3,9,43,52,32,8,8,14,27,9


# Clean the data

 - Drop TEAM, MATCHUP, and Score (+/-), 
     - We wont be using the columns as features in the ML model
 - Convert the DATE column data from an object to datetime data
 

In [94]:
# Drop columns that will not be used for ML models
# Drop TEAM, MATCHUP, and Score (+/-) for reg df
dropped_df = joined_df.drop(["TEAM","MATCHUP","+/-"], axis=1)

dropped_df.head(3)

,DATE,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF
0,01/13/2019,W,290,140,49,104,47.1,13,36,36.1,...,36,80.6,17,42,59,24,16,8,21,27
1,11/29/2018,W,265,131,47,90,52.2,15,38,39.5,...,24,91.7,7,30,37,25,5,6,14,23
2,02/13/2019,W,241,129,44,92,47.8,16,38,42.1,...,30,83.3,9,43,52,32,8,8,14,27


In [95]:
# Change DATE column data type from object to datetime
dropped_df["DATE"] = pd.to_datetime(dropped_df["DATE"],format="%m/%d/%Y")
dropped_df.head(3)

,DATE,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF
0,2019-01-13,W,290,140,49,104,47.1,13,36,36.1,...,36,80.6,17,42,59,24,16,8,21,27
1,2018-11-29,W,265,131,47,90,52.2,15,38,39.5,...,24,91.7,7,30,37,25,5,6,14,23
2,2019-02-13,W,241,129,44,92,47.8,16,38,42.1,...,30,83.3,9,43,52,32,8,8,14,27


In [96]:
# Check column data types
dropped_df.dtypes

DATE    datetime64[ns]
W/L             object
MIN              int64
PTS              int64
FGM              int64
FGA              int64
FG%            float64
3PM              int64
3PA              int64
3P%            float64
FTM              int64
FTA              int64
FT%            float64
OREB             int64
DREB             int64
REB              int64
AST              int64
STL              int64
BLK              int64
TOV              int64
PF               int64
dtype: object

In [97]:
# Count the number of wins and losses
dropped_df["W/L"].value_counts()

W    74
L    32
Name: W/L, dtype: int64

In [99]:
# Convert W/L column 
# Make W = 0 and L = 1
# Count the number of wins and losses to confirm conversion
# Convert object type to int64 datatype

dropped_df.loc[dropped_df['W/L'] == "W", "W/L"] = 0
dropped_df.loc[dropped_df['W/L'] == "L", "W/L"] = 1

dropped_df["W/L"] = pd.to_numeric(dropped_df["W/L"])
    
dropped_df["W/L"].value_counts()

0    74
1    32
Name: W/L, dtype: int64

In [100]:
# Check column data types
dropped_df.dtypes

DATE    datetime64[ns]
W/L              int64
MIN              int64
PTS              int64
FGM              int64
FGA              int64
FG%            float64
3PM              int64
3PA              int64
3P%            float64
FTM              int64
FTA              int64
FT%            float64
OREB             int64
DREB             int64
REB              int64
AST              int64
STL              int64
BLK              int64
TOV              int64
PF               int64
dtype: object

In [ ]:
# Drop null columns and rows
